<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных:" data-toc-modified-id="Обзор-данных:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обзор данных:</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**План**

1. Загрузить и подготовитье данные.
2. Обучить разные модели. 
3. Выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Обзор данных:

Загрузим датасет:

In [12]:
try:
    comments = pd.read_csv('C:/ya_pr/comments/toxic_comments.csv', index_col=[0])
except:
    comments = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])
    


Ознакомимся с данными:

In [13]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [14]:
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [15]:
comments.duplicated().sum()

0

 - Дубликатов нет.

Проверим баланс классов по целевому признаку:

In [16]:
comments.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

 - классы не сбалансированы. Учтем это при разбиении на тренировочную и тестовую выборки, а так же при обучении моделей.

### Подготовка данных

Удалим из текстов посторонние символы, числа и лишние пробелы, а так же приведем к нижнему регистру. Токенизируем и лемматизируем тексты. Для этого сделаем соответствующие функции:

Очистка текста от лишних символов:

In [17]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text).lower()
    return " ".join(text.split())

Получение POS-тегов NLTK и сопоставление с форматом, принятым лемматизатором wordnet:

In [18]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

Лемматизация с учетом POS-тегов (частей речи):

In [19]:
def lemmatize_pos(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemmatized_output

In [20]:
comments['lemm'] = comments['text'].apply(clear_text)
comments['lemm'] = comments['lemm'].apply(lemmatize_pos)

In [21]:
comments.head()

,text,toxic,lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Разобьем данные на тренировочную и тестовую выборки, с учетом выявленного ранее дисбаланса классов:

In [22]:
X_train, X_test, y_train, y_test = train_test_split(comments['lemm'],
                                                    comments['toxic'],
                                                    test_size=0.2,
                                                    random_state=123,
                                                    stratify=comments['toxic'])

Векторизуем тексты и очистим их от стоп-слов:

In [23]:
stopwords = set(nltk_stopwords.words('english'))

In [24]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,1), stop_words=stopwords,
               use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [25]:
X_train_vec = tf_idf_vec.fit_transform(X_train)

In [26]:
X_test_vec = tf_idf_vec.transform(X_test)

In [29]:
print(f"Тренировочная выборка: {X_train_vec.shape}")
print(f"Тестовая выборка: {X_test_vec.shape}")

Тренировочная выборка: (127433, 132685)
Тестовая выборка: (31859, 132685)


## Обучение

Функция для обучения модели:

In [30]:
def execute_model(model, param, features, target):
    cv_model = GridSearchCV(
        estimator=model,
        param_grid=param,
        cv=3,
        scoring='f1',
        n_jobs=-1,
        verbose=2)
    
    cv_model.fit(features, target)
    
    print('Лучший F1:',cv_model.best_score_)
    print('Лучшие параметры:',cv_model.best_params_)

### Логистическая регрессия

Набор параметров учитывает существенный дисбаланс классов:

In [19]:
lr_model = LogisticRegression()
params = {'C': [5,10,15],
         'class_weight': ['balanced','None']}
execute_model(lr_model, params, X_train_vec, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END .........................C=5, class_weight=balanced; total time=  37.1s
[CV] END .........................C=5, class_weight=balanced; total time=  35.8s
[CV] END .........................C=5, class_weight=balanced; total time=  38.0s
[CV] END .............................C=5, class_weight=None; total time=  37.4s
[CV] END .............................C=5, class_weight=None; total time=  37.2s
[CV] END .............................C=5, class_weight=None; total time=  34.8s
[CV] END ........................C=10, class_weight=balanced; total time=  39.7s
[CV] END ........................C=10, class_weight=balanced; total time=  36.8s
[CV] END ........................C=10, class_weight=balanced; total time=  37.6s
[CV] END ............................C=10, class_weight=None; total time=  39.4s
[CV] END ............................C=10, class_weight=None; total time=  37.9s
[CV] END ............................C=10, class_

### Дерево решений

In [20]:
dt_model = DecisionTreeClassifier()
params = {'max_depth': [60,80,100]}
execute_model(dt_model, params, X_train_vec, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END .......................................max_depth=60; total time=  22.3s
[CV] END .......................................max_depth=60; total time=  22.3s
[CV] END .......................................max_depth=60; total time=  22.6s
[CV] END .......................................max_depth=80; total time=  27.6s
[CV] END .......................................max_depth=80; total time=  28.0s
[CV] END .......................................max_depth=80; total time=  28.4s
[CV] END ......................................max_depth=100; total time=  33.0s
[CV] END ......................................max_depth=100; total time=  33.3s
[CV] END ......................................max_depth=100; total time=  33.7s
Лучший F1: 0.7158289728700611
Лучшие параметры: {'max_depth': 100}


In [21]:
dt_b_model = DecisionTreeClassifier()
params = {'max_depth': [60,80,100],
         'class_weight': ['balanced']}
execute_model(dt_b_model, params, X_train_vec, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ................class_weight=balanced, max_depth=60; total time=  32.0s
[CV] END ................class_weight=balanced, max_depth=60; total time=  31.8s
[CV] END ................class_weight=balanced, max_depth=60; total time=  31.6s
[CV] END ................class_weight=balanced, max_depth=80; total time=  38.1s
[CV] END ................class_weight=balanced, max_depth=80; total time=  39.1s
[CV] END ................class_weight=balanced, max_depth=80; total time=  37.7s
[CV] END ...............class_weight=balanced, max_depth=100; total time=  42.3s
[CV] END ...............class_weight=balanced, max_depth=100; total time=  43.5s
[CV] END ...............class_weight=balanced, max_depth=100; total time=  41.0s
Лучший F1: 0.641939214164111
Лучшие параметры: {'class_weight': 'balanced', 'max_depth': 100}


In [31]:
rf_model = RandomForestClassifier()
params = {'max_depth': [50,80]}
execute_model(rf_model, params, X_train_vec, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END .......................................max_depth=50; total time= 1.5min
[CV] END .......................................max_depth=50; total time= 1.5min
[CV] END .......................................max_depth=50; total time= 1.6min
[CV] END .......................................max_depth=80; total time= 2.5min
[CV] END .......................................max_depth=80; total time= 2.5min
[CV] END .......................................max_depth=80; total time= 2.7min
Лучший F1: 0.15801372410607387
Лучшие параметры: {'max_depth': 80}


In [32]:
rf_b_model = RandomForestClassifier()
params = {'max_depth': [50,80],
         'class_weight': ['balanced']}
execute_model(rf_b_model, params, X_train_vec, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END ................class_weight=balanced, max_depth=50; total time= 1.7min
[CV] END ................class_weight=balanced, max_depth=50; total time= 1.7min
[CV] END ................class_weight=balanced, max_depth=50; total time= 1.6min
[CV] END ................class_weight=balanced, max_depth=80; total time= 2.8min
[CV] END ................class_weight=balanced, max_depth=80; total time= 2.8min
[CV] END ................class_weight=balanced, max_depth=80; total time= 2.8min
Лучший F1: 0.5254520626366858
Лучшие параметры: {'class_weight': 'balanced', 'max_depth': 80}


## Выводы

Лучшие результаты на тренировочных выборках показала модель "Логистическая регрессия":

Лучший F1: 0.765

Лучшие параметры: {'C': 15, 'class_weight': 'None'}

 - отметим, что несмотря на дисбаланс классов, модель (логистическая регрессия) не учитывающая этот факт показала лучшие результаты. ("Случайный лес деревьев" наоборот - вариант с учетом дисбаланса классов показывает кратно лучшие результаты, но в любом случае они хуже чем у логистической регрессии.

Обучим лучшую модель на полной тренировочной выборке и посчитаем метрику качества на тестовой выборке:

In [33]:
model = LogisticRegression(C=15)
model.fit(X_train_vec, y_train)
predictions = model.predict(X_test_vec)
f1_test = f1_score(y_test,predictions)

print(f'F1 на тестовой выборке: {f1_test:.3f}')


F1 на тестовой выборке: 0.783


 - Требуемый в задаче размер метрики качества F1 (f1>0.75) достигнут на модели "Логистическая регрессия": F1 = 0.783